# Script to Fetch Google Trends data for a keyword

In [ ]:
pip install pytrends

In [ ]:
from pytrends.request import TrendReq
import pandas as pd
pytrends=TrendReq()

In [ ]:
import datetime as dt

In [ ]:
# method to fetch trends data from google
def fetchTrendsOverTime(keyword,geoList):
  dict_trends={}
  for g in geoList:
      dict_trends[g]={}
      pytrends.build_payload([keyword],timeframe='today 3-m',geo=g) #cat parameters can be applied to filter by category in search
      result=pytrends.interest_over_time()
      dict_trends[g]=result.copy()
      if not dict_trends[g].empty:
        dict_trends[g] = dict_trends[g].drop(labels=['isPartial'],axis='columns')

  #converting dict to df
  dict_trends_1={g : pd.DataFrame(k) for g,k in dict_trends.items()}
  data_df=pd.concat(dict_trends_1,axis=1)
  return data_df,result


In [ ]:
keyword='Haus kaufen'   # keyword for search
geoList=['DE','CH']     # countries abbreviation list for search
trend_df,result=fetchTrendsOverTime(keyword,geoList) #method call
trend_df

,DE,CH
,Haus kaufen,Haus kaufen
date,,
2021-07-18,78,65
2021-07-19,80,36
2021-07-20,68,28
2021-07-21,75,43
2021-07-22,62,44
...,...,...
2021-10-12,69,51
2021-10-13,75,49


In [ ]:
trend_df=trend_df.reset_index()

In [ ]:
trend_df['Years']=trend_df['date'].dt.year
trend_df['Quarter']=trend_df['date'].dt.quarter
trend_df['Day']=trend_df['date'].dt.day
trend_df

,date,DE,CH,Years,Quarter,Day
,,Haus kaufen,Haus kaufen,,,
0,2021-07-18,78,65,2021,3,18
1,2021-07-19,80,36,2021,3,19
2,2021-07-20,68,28,2021,3,20
3,2021-07-21,75,43,2021,3,21
4,2021-07-22,62,44,2021,3,22
...,...,...,...,...,...,...
86,2021-10-12,69,51,2021,4,12
87,2021-10-13,75,49,2021,4,13
88,2021-10-14,71,39,2021,4,14


**Including Timestamp of runtime**

In [ ]:
from datetime import datetime
trend_df['TimeStamp'] = datetime.now().replace(microsecond=0)
trend_df

,DE,CH,TimeStamp
,Haus kaufen,Haus kaufen,
date,,,
2021-06-27,94,36,2021-09-27 09:25:04
2021-06-28,74,40,2021-09-27 09:25:04
2021-06-29,59,26,2021-09-27 09:25:04
2021-06-30,68,50,2021-09-27 09:25:04
2021-07-01,72,41,2021-09-27 09:25:04
...,...,...,...
2021-09-20,68,30,2021-09-27 09:25:04
2021-09-21,79,36,2021-09-27 09:25:04


# Calculating Moving Average

In [ ]:
#creating a copy of the dataset from the previous step
moving_average_df=trend_df.copy()

#moving averages for every country data
for g in geoList:
  var_name=g+'_Moving_average_21'
  moving_average_df[var_name]=moving_average_df[g][keyword].rolling(21).mean()  #moving average for 21 days interval

  #uncomment to avoid null values in the beginning of movig average values
  #moving_average_df[var_name]=moving_average_df[g][kw_list].rolling(21,min_periods=1).mean()

  moving_average_df[var_name]=moving_average_df[var_name].apply(lambda x:round(x,2))  #rounding off moving average to 2 decimal points for simplicity

moving_average_df

,DE,CH,TimeStamp,DE_Moving_average_21,CH_Moving_average_21
,Haus kaufen,Haus kaufen,,,
date,,,,,
2021-06-27,94,36,2021-09-27 09:25:04,NaN,NaN
2021-06-28,74,40,2021-09-27 09:25:04,NaN,NaN
2021-06-29,59,26,2021-09-27 09:25:04,NaN,NaN
2021-06-30,68,50,2021-09-27 09:25:04,NaN,NaN
2021-07-01,72,41,2021-09-27 09:25:04,NaN,NaN
...,...,...,...,...,...
2021-09-20,68,30,2021-09-27 09:25:04,73.38,40.52
2021-09-21,79,36,2021-09-27 09:25:04,74.00,40.43
